<a href="https://colab.research.google.com/github/eynomr/CSP-Sumplete/blob/main/CSP_sumplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aimacode/aima-python.git
!git clone https://github.com/aimacode/aima-data /content/aima-python/aima-data

fatal: destination path 'aima-python' already exists and is not an empty directory.
fatal: destination path '/content/aima-python/aima-data' already exists and is not an empty directory.


In [ ]:
#TODO Step One: Figure out: set of variables, domains for each variable, constraint graph representation, applicable constraints
# I think the set of variables could be each cell in the grid.
# Alternatively, the variables could be the rows and columns.
# Domain would be each value in the row and column.
# Constraint obviously is that each row and column adds up to its prescribed value.

# AIMA code
#https://github.com/aimacode/aima-python/blob/master/csp.ipynb
# Found another good source of info on this topic
#https://simpleai.readthedocs.io/en/latest/constraint_satisfaction_problems.html

In [ ]:
import sys
sys.path.insert(0, "/content/aima-python")
from itertools import combinations
from csp import *

In [ ]:
def revise(csp, Xi, Xj, removals):
    """Return true if we remove a value."""
    revised = False
    for x in csp.curr_domains[Xi][:]:
        # If Xi=x conflicts with Xj=y for every possible y, eliminate Xi=x
        if all(not csp.constraints(Xi, x, Xj, y) for y in csp.curr_domains[Xj]):
            csp.prune(Xi, x, removals)
            revised = True
    return revised

def AC3(csp, queue=None, removals=None, arc_heuristic=dom_j_up):
    """[Figure 6.3]"""
    if queue is None:
        queue = {(Xi, Xk) for Xi in csp.variables for Xk in csp.neighbors[Xi]}
    csp.support_pruning()
    queue = arc_heuristic(csp, queue)
    while queue:
        (Xi, Xj) = queue.pop()
        if revise(csp, Xi, Xj, removals):
            if not csp.curr_domains[Xi]:
                return False
            for Xk in csp.neighbors[Xi]:
                if Xk != Xj:
                    queue.add((Xk, Xi))
    return True

In [ ]:
# goal = {
#     #row
#     (4, 7, 5): 7,
#     (4, 2, 6): 6,
#     (6, 2, 2): 4,

#     #column
#     (4, 4, 6): 0,
#     (7, 2, 2): 9,
#     (5, 6, 2): 8,
# }
# goal = {
#     #row
#     (2, 9, 9): 11,
#     (5, 6, 5): 5,
#     (8, 7, 9): 17,

#     #column
#     (2, 5, 8): 10,
#     (9, 6, 7): 0,
#     (9, 5, 9): 23,
# }
goal = {
    #row
    (3, 4, 9, 3, 5, 1): 15,
    (6, 2, 7, 4, 3, 6): 9,
    (2, 5, 9, 6, 4, 5): 9,
    (9, 5, 2, 1, 5, 2): 20,
    (3, 1, 4, 6, 3, 1): 15,
    (1, 3, 9, 4, 6, 9): 32,

    #column
    (3, 6, 2, 9, 3, 1): 13,
    (4, 2, 5, 5, 1, 3): 16,
    (9, 7, 9, 2, 4, 9): 22,
    (3, 4, 6, 1, 6, 4): 15,
    (5, 3, 4, 5, 3, 6): 23,
    (1, 6, 5, 2, 1, 9): 11,
}

# variables = [
#     #row
#     (2, 9, 9),
#     (5, 6, 5),
#     (8, 7, 9),

#     #column
#     (2, 5, 8),
#     (9, 6, 7),
#     (9, 5, 9),
# ]
variables = [
    #row
    (3, 4, 9, 3, 5, 1),
    (6, 2, 7, 4, 3, 6),
    (2, 5, 9, 6, 4, 5),
    (9, 5, 2, 1, 5, 2),
    (3, 1, 4, 6, 3, 1),
    (1, 3, 9, 4, 6, 9),

    #column
    (3, 6, 2, 9, 3, 1),
    (4, 2, 5, 5, 1, 3),
    (9, 7, 9, 2, 4, 9),
    (3, 4, 6, 1, 6, 4),
    (5, 3, 4, 5, 3, 6),
    (1, 6, 5, 2, 1, 9),
]

half_len = int(len(variables)/2)
rows = variables[:half_len]
columns = variables[half_len:]


In [ ]:
# Helper functions

# For domain
def all_combos(lst):
  # All possible values for rows and columns that sums up to the goal of that row or column.
  all_combinations = set()
  if sum(lst) == goal[tuple(lst)]:
    all_combinations.add(tuple(lst))
  for r in range(1, len(lst) + 1):
      for indices in combinations(range(len(lst)), r):
          new_list = lst.copy()
          for i in indices:
              new_list[i] = 0
          if sum(new_list) == goal[tuple(lst)]:
            all_combinations.add(tuple(new_list))
  return sorted(all_combinations)

# for neighbors
def get_neighbors(variables, domain):
  # neighbors are all columns are row that intersect with a cell that is set to 0 (which means it was crossed) since those are the only ones that share the constraint.
  neighbors = {}
  hal_len = int(len(variables)/2)
  rs = variables[:hal_len]
  cs = variables[hal_len:]

  for row in rs:
    var_domain = domain[row]
    neighbors[row] = []
    for d in var_domain:
      zero_indices = [index for index, value in enumerate(d) if value == 0]
      for index in zero_indices:
        neighbors[row].append(columns[index])

  for col in cs:
    var_domain = domain[col]
    neighbors[col] = []
    for d in var_domain:
      zero_indices = [index for index, value in enumerate(d) if value == 0]
      for index in zero_indices:
        neighbors[col].append(rows[index])

  return neighbors

# Contraint function
def sum_constraint(row, row_val, col, col_val):
  # checks if the intersecting cell of neighboring variables is of the same value.

  if row in rows:
    row_index = rows.index(row)
  else:
    row_index = columns.index(row)
  if col in columns:
    col_index = columns.index(col)
  else:
    col_index = rows.index(col)

  return row_val[col_index] == col_val[row_index]

In [ ]:
domain = {}
for var in variables:
  domain[var] = all_combos(list(var))

neighbors = get_neighbors(variables, domain)

print("Domain:")
for key, val in domain.items():
  print(key, " : ", val)

print('---------------------')

print("Neighbors:")
for key, val in neighbors.items():
  print(key, " : ", val)


Domain:
(3, 4, 9, 3, 5, 1)  :  [(0, 0, 9, 0, 5, 1), (3, 0, 9, 3, 0, 0), (3, 4, 0, 3, 5, 0)]
(6, 2, 7, 4, 3, 6)  :  [(0, 0, 0, 0, 3, 6), (0, 2, 0, 4, 3, 0), (0, 2, 7, 0, 0, 0), (6, 0, 0, 0, 3, 0)]
(2, 5, 9, 6, 4, 5)  :  [(0, 0, 0, 0, 4, 5), (0, 0, 9, 0, 0, 0), (0, 5, 0, 0, 4, 0)]
(9, 5, 2, 1, 5, 2)  :  [(9, 5, 0, 1, 5, 0)]
(3, 1, 4, 6, 3, 1)  :  [(0, 1, 4, 6, 3, 1), (3, 1, 4, 6, 0, 1)]
(1, 3, 9, 4, 6, 9)  :  [(1, 3, 9, 4, 6, 9)]
(3, 6, 2, 9, 3, 1)  :  [(0, 0, 0, 9, 3, 1), (3, 0, 0, 9, 0, 1), (3, 6, 0, 0, 3, 1)]
(4, 2, 5, 5, 1, 3)  :  [(0, 2, 5, 5, 1, 3), (4, 2, 5, 5, 0, 0)]
(9, 7, 9, 2, 4, 9)  :  [(0, 0, 9, 0, 4, 9), (0, 7, 0, 2, 4, 9), (0, 7, 9, 2, 4, 0), (9, 0, 0, 0, 4, 9), (9, 0, 9, 0, 4, 0), (9, 7, 0, 2, 4, 0)]
(3, 4, 6, 1, 6, 4)  :  [(0, 4, 0, 1, 6, 4), (0, 4, 6, 1, 0, 4), (3, 0, 6, 0, 6, 0)]
(5, 3, 4, 5, 3, 6)  :  [(5, 0, 4, 5, 3, 6), (5, 3, 4, 5, 0, 6)]
(1, 6, 5, 2, 1, 9)  :  [(0, 0, 0, 2, 0, 9), (0, 6, 5, 0, 0, 0), (1, 0, 0, 0, 1, 9)]
---------------------
Neighbors:
(3, 4, 9, 3

In [ ]:
csp = CSP(variables=variables, domains=domain, neighbors=neighbors, constraints=sum_constraint)

In [ ]:
removals = []
AC3(csp, removals=removals)

True

In [ ]:
# what was removed from the domains
removals

[((2, 9, 9), (2, 9, 0)), ((5, 6, 5), (5, 0, 0))]

In [ ]:
result = backtracking_search(csp)
print(result)

{(3, 4, 9, 3, 5, 1): (0, 0, 9, 0, 5, 1), (6, 2, 7, 4, 3, 6): (0, 2, 0, 4, 3, 0), (2, 5, 9, 6, 4, 5): (0, 5, 0, 0, 4, 0), (9, 5, 2, 1, 5, 2): (9, 5, 0, 1, 5, 0), (3, 1, 4, 6, 3, 1): (3, 1, 4, 6, 0, 1), (1, 3, 9, 4, 6, 9): (1, 3, 9, 4, 6, 9), (3, 6, 2, 9, 3, 1): (0, 0, 0, 9, 3, 1), (4, 2, 5, 5, 1, 3): (0, 2, 5, 5, 1, 3), (9, 7, 9, 2, 4, 9): (9, 0, 0, 0, 4, 9), (3, 4, 6, 1, 6, 4): (0, 4, 0, 1, 6, 4), (5, 3, 4, 5, 3, 6): (5, 3, 4, 5, 0, 6), (1, 6, 5, 2, 1, 9): (1, 0, 0, 0, 1, 9)}
